- Import Package

In [1]:
# 載入所需套件
import requests, numpy as np, pandas as pd, time, random
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from IPython.display import clear_output
import urllib.parse, time
from datetime import datetime
from IPython.display import clear_output
import os

In [ ]:
id_list = []
for i, j in zip(indie,pd.read_excel('Game_info.xlsx')['Indie_name'].values[:20].tolist()):
    if str(i)+'.csv' in os.listdir('Selenium Crawling/'):
        print(j)
        print(str(i)+'.csv')
        print(pd.read_csv('Selenium Crawling/'+str(i)+'.csv').shape[0])
    elif str(i)+'error.csv' in os.listdir('Selenium Crawling/'):
        print(j)
        print(str(i)+'error.csv')
    else:
        print(j)
        print(str(i))
    print('---'*15)

In [ ]:
sport = pd.read_excel('Game_info.xlsx')['Sports_id'].values[:20].tolist()
strategy = pd.read_excel('Game_info.xlsx')['Strategy_id'].values[:20].tolist()
simulation = pd.read_excel('Game_info.xlsx')['Simulation_id'].values[:20].tolist()
RPG = pd.read_excel('Game_info.xlsx')['RPG_id'].values[:20].tolist()
indie = pd.read_excel('Game_info.xlsx')['Indie_id'].values[:20].tolist()
action = pd.read_excel('Game_info.xlsx')['Action_id'].values[:21].tolist()
adventure = pd.read_excel('Game_info.xlsx')['Adventure_id'].values[:20].tolist()
for i in ['920210']:
    try:
        print(i, pd.read_csv('Selenium Crawling/'+str(i)+'.csv').shape[0])
    except:
        print(i)

- Crawling reviews using Selenium

In [3]:
# read game id 
id_list = ['261550',]

# crawling
for id in id_list:
    url = f'https://steamcommunity.com/app/{id}/reviews/?browsefilter=mostrecent&snr=1_5_100010_&p=1&filterLanguage=english'
    driver = webdriver.Chrome('chromedriver')
    driver.get(url) 
    actions = ActionChains(driver)
    html_txt, arrow_down_count = '', 0
    try:
        elements = driver.find_elements(By.XPATH, '//*[@id="age_gate_btn_continue"]/span')
        for element in elements:
            element.click()
    except:
        pass
    while True:
        arrow_down_count += 1
        actions.send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(0.1)
        if arrow_down_count % 500 == 0:
            clear_output()
            print(f'now is crawling game {id}, with {arrow_down_count} page down')
            print(len(html_txt),len(driver.page_source))
            if len(html_txt) == len(driver.page_source):
                print(f'stop at {arrow_down_count}th arrow down')
                html_txt = driver.page_source
                break
            if '張貼於：2021 年 12 月' in html_txt:
                html_txt = driver.page_source
                break               
            html_txt = driver.page_source

    # find the elements
    html_txt = driver.page_source
    soup = BeautifulSoup(html_txt, 'html.parser')
    review_list = soup.find_all('div', {'class':'apphub_CardTextContent'})
    helpfulness_list = soup.find_all('div', {'class':'found_helpful'})
    print(len(review_list),len(helpfulness_list))

    # export result
    store_dict = {'review':[], 'helpful':[]}
    for ith in range(len(review_list)):
        store_dict['review'].append(review_list[ith].text)
        store_dict['helpful'].append(helpfulness_list[ith].text)
    pd.DataFrame(store_dict).to_csv('Selenium Crawling/'+str(id) + '.csv', index = False)

now is crawling game 261550, with 9000 page down
43052553 45845192


KeyboardInterrupt: 

In [ ]:
html_txt = driver.page_source
soup = BeautifulSoup(html_txt, 'html.parser')
review_list = soup.find_all('div', {'class':'apphub_CardTextContent'})
helpfulness_list = soup.find_all('div', {'class':'found_helpful'})
print(len(review_list),len(helpfulness_list))

# export result
store_dict = {'review':[], 'helpful':[]}
for ith in range(len(review_list)):
    store_dict['review'].append(review_list[ith].text)
    store_dict['helpful'].append(helpfulness_list[ith].text)
pd.DataFrame(store_dict).to_csv('Selenium Crawling/'+str(id) + '.csv', index = False)